In [1]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
df = pd.read_csv('connection_data//Connections_Data.csv')

In [7]:
df.groupby("Game ID").head()

,Game ID,Puzzle Date,Word,Group Name,Group Level,Starting Row,Starting Column
0,1,2023-06-12,SNOW,WET WEATHER,0,1,1
1,1,2023-06-12,LEVEL,PALINDROMES,3,1,2
2,1,2023-06-12,SHIFT,KEYBOARD KEYS,2,1,3
3,1,2023-06-12,KAYAK,PALINDROMES,3,1,4
4,1,2023-06-12,HEAT,NBA TEAMS,1,2,1
...,...,...,...,...,...,...,...
14432,953,2025-12-03,LOBBY,PETITION (FOR),2,1,1
14433,953,2025-12-03,PRESS,PETITION (FOR),2,1,2
14434,953,2025-12-03,ROCKET,"""___ MAN"" SONGS FROM THE '70S",3,1,3
14435,953,2025-12-03,RECEPTION,SOCIAL GATHERING,1,1,4


In [8]:
games = []

for game_id, g in df.groupby("Game ID"):
    words = g["Word"].tolist()
    labels = g["Group Level"].tolist()

    if len(words) != 16:
        continue

    games.append({"words": words, "labels": labels})

In [9]:
print(games[0]['words'])
print(games[0]['labels'])

['SNOW', 'LEVEL', 'SHIFT', 'KAYAK', 'HEAT', 'TAB', 'BUCKS', 'RETURN', 'JAZZ', 'HAIL', 'OPTION', 'RAIN', 'SLEET', 'RACECAR', 'MOM', 'NETS']
[0, 3, 2, 3, 1, 2, 1, 2, 1, 0, 2, 0, 0, 3, 3, 1]
